# MNIST Tensorflow 2.0 실습

In [126]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기

In [127]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()

In [128]:
(X,y) = datasets. load_digits(return_X_y=True)
X = X.astype(np.float32)
y = y.astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size=0.2, random_state=1)

# Hidden Layer 숫자를 지정

In [129]:
n_inputs = 64
n_hidden_1 = 25
n_hideen_2 = 25
n_outputs = 10

# 신경망 클래스 빌드

In [167]:
class NNLayer(object):
    def __init__(self, n_inputs, n_neurons, activation=None):
        init = tf.random.normal((n_inputs, n_neurons),
                               stddev=2 / np.sqrt(n_inputs))
        self.W = tf.Variable(init, name = 'kernel')
        self.b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        self.activation = activation
        
    def __call__(self, X):
        Z = tf.matmul(X,self.W) + self.b
        if self.activation is not None:
            return self.activation(Z)
        else:
            return Z
        
    def trainable_variables(self):
        return [self.W, self.b]

In [168]:
class Model (object):
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, X):
        for layer in self.layers:
            X = layer(X)
        return X
    
    def trainable_variables(self):
        variables= []
        for layer in self.layers:
            variables.extend(layer.trainable_variables())
        return variables

In [169]:
# Loss Function

loss_function = tf.nn.sparse_softmax_cross_entropy_with_logits

## Hidden Layers 만들기

In [170]:
model = Model([
    NNLayer(n_inputs, n_hidden_1, activation=tf.nn.relu),
    NNLayer(n_hidden_1, n_hideen_2, activation=tf.nn.relu),
    NNLayer(n_hideen_2, n_outputs)
])

In [171]:
def loss(model, X, y_true):
    y_pred = model(X)
    return tf.reduce_mean(tf.dtypes.cast(loss_function(labels=y_true, logits=y_pred),
                                       tf.float32, name='loss'))

In [172]:
l = loss(model, X_train, y_train)
print('Testing loss', l)

Testing loss tf.Tensor(87.61525, shape=(), dtype=float32)


In [173]:
# GD Background
def grad(model, X, y_true) :
    with tf.GradientTape() as tape :
        loss_value = loss(model, X, y_true)
    return loss_value, tape.gradient(loss_value, model.trainable_variables())

In [174]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)

In [175]:
loss_value, grads = grad(model, X_train, y_train)
print('Step : {}, Initial loss: {}'.format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

Step : 0, Initial loss: 87.61524963378906
